# Short template to create 3D visualisations with **cubeX3D**

**cubeX3D** provides a straightforward method to create 3D visualisations of datacubes, with the possibility of customising the visualisation in many ways. This template describes how to create different 3D visualisations with the **cubeX3D** package.

Apart from the provided examples, many other visualisations can be created by combining the different parameters available in the functions *vis.prep_one*, *vis.prep_mult*, *vis.prep_overlay*. See the documentation of each function for more information.

Even though those functions are recommended to create the visualisations, they can also be created from scratch with only the vis.Cube object and functions in the writers submodule.

**REMEMBER THAT THE VISUALISATIONS MUST BE OPENED THROUGH A LOCAL SERVER, NOT DIRECTLY FROM THE FILE SYSTEM.**

In [1]:
from visl3d import visual as vis
import astropy.units as u

## Default visualisation

In the simplest case, a 3D visualisation of an entire datacube can be created with just giving the name of a FITS file as input:

In [2]:
cube = vis.prep_one('../../HCG16.fits')

vis.createX3D(cube, 'HCG16_default.x3d')
vis.createHTML(cube, 'HCG16_default.html')

The function *vis.prep_one* returns a Cube object with all the information needed to create a 3D visualisation.<br>
Then, the X3D file and the HTML file are created with *vis.createX3D* and *vis.createHTML* respectively.

## Custom visualisation, single spectral line

Several parameters can be introduced in *vis.prep_one* to customise the visualisation:

In [3]:
lims = [[32.2*u.deg,32.7*u.deg],
        [-10.4*u.deg,-10.2*u.deg],
        [3800*u.km/u.s,4000*u.km/u.s]]

galaxies = ['HCG16a','HCG16b', 'HCG16c', 'HCG16d']

In [4]:
cube = vis.prep_one('../../HCG16.fits', lims=lims, unit='rms', isolevels=[3,4,5,7,10],
                    colormap='magma_r', image2d='DSS2 Blue', galaxies=galaxies)

vis.createX3D(cube, 'HCG16_custom.x3d')
vis.createHTML(cube, 'HCG16_custom.html')

## Multiple spectral lines

Visualisations with several subcubes can be created to represent different spectral lines.
In this case the spectral_lims parameter is mandatory.

In [5]:
spectral_lims = [[3600*u.km/u.s,3700*u.km/u.s],
                 [3800*u.km/u.s,3900*u.km/u.s]]

spatial_lims = [[[32.3*u.deg,32.5*u.deg],[-10.4*u.deg,-10.3*u.deg]],
                [[32.6*u.deg,32.7*u.deg],[-10.2*u.deg,-10.1*u.deg]]]

lines = ['First', 'Second']

In [6]:
cube = vis.prep_mult('../../HCG16.fits', spectral_lims=spectral_lims, spatial_lims=spatial_lims,
                     lines = lines, unit='rms')

vis.createX3D(cube, 'HCG16_mult.x3d')
vis.createHTML(cube, 'HCG16_mult.html')

## Spectral line overlay

Instead of plotting different spectral lines separately, they can be overlaid in the same volume.<br>
The subcubes can be definead as with *vis.prep_mult* or with the lines parameter as a dictionary.

In [7]:
lines = {'Line 1': (3600*u.km/u.s, 200*u.km/u.s), 'Line 2': (3850*u.km/u.s, 20)}

spatial_lims = [[[32.3*u.deg,32.5*u.deg],[-10.4*u.deg,-10.3*u.deg]],
                [[32.6*u.deg,32.7*u.deg],[-10.25*u.deg,-10.1*u.deg]]]

In [8]:
cube = vis.prep_overlay('../../HCG16.fits', lines=lines, spatial_lims=spatial_lims,
                        unit='percent', colormap=['Reds', 'Blues'])

vis.createX3D(cube, 'HCG16_overlay.x3d')
vis.createHTML(cube, 'HCG16_overlay.html')

## Manual method

We can make the whole process manually. We can load and prepare data without *vis.prep_one*,<br>
*vis.prep_mult* or *vis.prep_overlay* directly with e.g. Astropy or Spectral-Cube.

In [2]:
from spectral_cube import SpectralCube
import numpy as np
from cubeX3D import misc
from scipy.stats import norm
from astropy.coordinates import SkyCoord

# Read the fits file and save header
data = SpectralCube.read("../../HCG16.fits")
cubehead = data.header
nz, ny, nx = data.shape
print(data.shape)
dra, ddec, dv = cubehead['CDELT1'], cubehead['CDELT2'], cubehead['CDELT3']
units = [cubehead['BUNIT'], cubehead['CUNIT1'], cubehead['CUNIT2'], cubehead['CUNIT3']]
mags = [cubehead['BTYPE'], cubehead['CTYPE1'], cubehead['CTYPE2'], cubehead['CTYPE3']]
delta = (np.abs(dra), np.abs(ddec), np.abs(dv))
print(delta)

# Limits to crop the cube, no HI there. Subtract 1 to wanted number, e.g. nz-1.
limx = [25, nx-16]
limy = [20, ny-21]
limz = [10, nz-31]

# Calculate coordinates from limits
ralim = data.spatial_coordinate_map[1][0,:][limx][::int(np.sign(cubehead["CDELT1"]))]
ramean = np.mean(ralim)
declim = data.spatial_coordinate_map[0][:,0][limy][::int(np.sign(cubehead["CDELT2"]))]
decmean = np.mean(declim)
vlim = data.spectral_axis[limz][::int(np.sign(dv))]
vmean = np.mean(vlim).to_value()

coords = np.array([ralim, declim, vlim])
print(coords)

# create cropped array
data = data.unmasked_data[limz[0]:limz[1]+1,limy[0]:limy[1]+1,limx[0]:limx[1]+1].to_value()
# Slice data, transpose to (ra,dec,v) and flip axes if needed (see if step in FITS header is + or -)
data = misc.transpose(data, (dra,ddec,dv))

# calculate rms of data from negative values in the cube
_, rms = norm.fit(np.hstack([data[data<0].flatten(),-data[data<0].flatten()]))
print(rms)

# Divide the cube by the RMS
data = data/rms
units[0] = 'RMS'

isolevels = np.array([3, 3.5, 4, 4.5, 5, 6, 7, 8 ,9, 10, 15])

galaxies = ['HCG16a','HCG16b', 'HCG16c', 'HCG16d', 'NCG848']
trans = (2000/nx, 2000/ny, 2000/nz)
galdict = misc.get_galaxies(galaxies, coords, units, cubehead['OBJECT'], delta, trans)

color = misc.create_colormap('CMRmap_r', isolevels)

pixels = 1000
co = SkyCoord(ra=np.mean(coords[0])*u.Unit(units[1]), dec=np.mean(coords[1])*u.Unit(units[2]))
co = co.to_string('hmsdms')
imcol, img_shape, _ = misc.get_imcol(position=co, survey='DSS2 Blue',pixels=f'{pixels}',
        coordinates='J2000', width=np.diff(coords[0])[0]*u.Unit(units[1]),
        height=np.diff(coords[1])[0]*u.Unit(units[2]))
image2d = imcol, img_shape

cube = vis.Cube(l_cubes=[data], name=cubehead['OBJECT'], coords=coords, units=units,
                 mags=mags, l_colors=[color], rms=rms, image2d=image2d, delta=delta,
                 galaxies=galdict, l_isolevels=[isolevels])

(133, 280, 315)
(0.00166666666667, 0.00166666666667, 5513.42184924)
[[ 3.23056917e+01  3.27701427e+01]
 [-1.04694518e+01 -1.00711173e+01]
 [ 3.65003088e+06  4.15726569e+06]]
0.00038725222


Now that we have defined the Cube object, we can create the X3D and HTML files with the *writers* submodule.<br>
The previous step can be done with *vis.prep_one*, *vis.prep_mult* or *vis.prep_overlay* and still create the files with the *writers* submodule.

In [3]:
from visl3d import writers

x3d = writers.WriteX3D('HCG16_manual.x3d', cube)
x3d.make_layers()
x3d.make_outline()
x3d.make_galaxies()
x3d.make_image2d()
x3d.make_ticklines()
x3d.make_animation()
x3d.make_labels()

We can add functionalities that are not implemented by *vis.createX3D* and *vis.createHTML*,<br>
such as adding a marker directly in the X3D model.

In [4]:
# variables needed to create markers
delta = cube.delta
nx, ny, nz = cube.l_cubes[0].shape
trans = [2000/nx, 2000/ny, 2000/nz]

In [5]:
# coordinates to create markers, in units of the cube.
tub = [np.array([[32.4, -10.1, 3850000], [32.6, -10.2, 4050000]]),
          np.array([[32.5, -10.3, 3900000], [32.7, -10.4, 4000000], [32.4, -10.2, 4100000]])]
sph = [[32.35,-10.15,3820000],[32.45,-10.45,3910000]]
box = [[32.4,-10.2,4100000],[32.6,-10.4,3900000]]
con = [[32.5,-10.3,3950000],[32.7,-10.1,4050000]]

In [6]:
# create markers in the x3d model and close.
x3d.make_markers(geom='tube', points=tub, shape=[50,10], delta=delta,
                 trans=trans, color=['1 0 0','0 1 0'])
x3d.make_markers(geom='sphere', points=sph, shape=[50,40], delta=delta,
                 trans=trans, color=['1 0 1','0 1 1'])
x3d.make_markers(geom='box', points=box, shape=[[100,50,50],[40,40,40]], delta=delta,
                 trans=trans, color=['1 1 0','0 0 1'])
x3d.make_markers(geom='cone', points=con, shape=[[60,200],[40,100]], delta=delta,
                 trans=trans, color=['0 1 1','1 1 0'])
x3d.close()

We also have to define give the coordinates of the markers in the HTML file to be able to interact with them.<br>
These are needed in *func_pymarkers()*, *func_buttons()* and *func_scalev()*.

Another functionality is adding custom viewpoints to give a better perspective of certain features in the data.<br>
This is done by adding points in *viewpoints()* and names in *buttons()*.

In [7]:
html = writers.WriteHTML('HCG16_manual.html', cube)
html.func_layers()
html.func_galaxies()
html.func_gallab()
html.func_grids()
html.func_axes()
html.func_animation()
html.func_pymarkers(tube=tub, sphere=sph, box=box, cone=con)
html.start_x3d()
html.viewpoints(point=[[1,1,1],[-2,1,0]])
html.close_x3d('HCG16_manual.html')
html.buttons(tube=tub, sphere=sph, box=box, cone=con, viewpoint=['Tidal tail', 'HII region'])
html.func_galsize()
html.func_image2d()
html.func_move2dimage()
html.func_scalev(tube=tub, sphere=sph, box=box, cone=con, delta=delta, trans=trans)
html.func_markers()
html.func_background()
html.func_colormaps()
html.close_html()